In [1]:
import pandas as pd

# read data

In [2]:
# read scraped data from interlegis city information

scrape_data = '/Users/liviaclarete/gdrive/code/mestrado_iesp/links_interlegis/links_interlegis/spiders/interlegis_data.csv'
df = pd.read_csv(scrape_data)

In [3]:
# test if the data was correctly read
print(df.shape)

df.head()

(5704, 4)


,title,url_camara,address,link
0,Câmara Municipal de Almino Afonso,"mailto:cmaa@brisanet.com.br,javascript:void(0);","Endereço:,Rua Antônio Carlos, nº 44 ,Centro,Al...",http://www.interlegis.leg.br/institucional/cas...
1,Câmara Municipal de Alta Floresta,"http://www.camaraaltafloresta.mt.gov.br/,mailt...","Endereço:,Av. Ariosto da Riva, nº 2349,Centro....",http://www.interlegis.leg.br/institucional/cas...
2,Câmara Municipal de Alpinópolis,"http://www.camaracapinopolis.mg.gov.br/,mailto...","Endereço:,Rua Professor Telles, nº 335,São Ben...",http://www.interlegis.leg.br/institucional/cas...
3,Câmara Municipal de Alpestre,"mailto:legislativoalpestre@yahoo.com.br,javasc...","Endereço:,Avenida Farrapos, nº 350,Centro,Alpe...",http://www.interlegis.leg.br/institucional/cas...
4,Câmara Municipal de Altair,"mailto:camaramunicipalaltair@hotmail.com,javas...","Endereço:,Rua 04, nº 240,Centro,Altair,São Pau...",http://www.interlegis.leg.br/institucional/cas...


# feature extraction

In [4]:
# extract street number, email,url and uf from scraped data
import re
def search_number(col):
    regex_email = '[nº]..[0-9]{1,7}'
    col_n = re.findall(regex_email, col)
    col_n_clean = "".join(col_n)
    col_n_clean = col_n_clean.replace(
        ' ','').replace('nº', '').replace('º', '').replace('n°', '')
    return(col_n_clean)

def search_email(col):
    regex_email = r'[\w\.-]+@[\w\.-]+'
    col_email = re.findall(regex_email, col)
    col_email_clean = "".join(col_email)
    return(col_email_clean)

def search_url(col):
    regex_email = r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    col_url = re.findall(regex_email, col)
    col_url_clean = "".join(col_url)
    return(col_url_clean)

def search_uf(col):
    regex_uf = r"[^,]*.CEP"
    col_uf = re.findall(regex_uf, col)
    col_uf_clean = "".join(col_uf)
    return(col_uf_clean)

In [5]:
# add new column with the state code

# copy dict from https://gist.github.com/ricardobarantini/5665214

uf_dict = {
    'AC': 'Acre',
    'AL': 'Alagoas',
    'AP': 'Amapá',
    'AM': 'Amazonas',
    'BA': 'Bahia',
    'CE': 'Ceará',
    'DF': 'Distrito Federal',
    'ES': 'Espírito Santo',
    'GO': 'Goiás',
    'MA': 'Maranhão',
    'MT': 'Mato Grosso',
    'MS': 'Mato Grosso do Sul',
    'MG': 'Minas Gerais',
    'PA': 'Pará',
    'PB': 'Paraíba',
    'PR': 'Paraná',
    'PE': 'Pernambuco',
    'PI': 'Piauí',
    'RJ': 'Rio de Janeiro',
    'RN': 'Rio Grande do Norte',
    'RS': 'Rio Grande do Sul',
    'RO': 'Rondônia',
    'RR': 'Roraima',
    'SC': 'Santa Catarina',
    'SP': 'São Paulo',
    'SE': 'Sergipe',
    'TO': 'Tocantins'
}

# swift postion from key to values
uf_dict_2 = {y:x for x,y in uf_dict.items()}


# download uf codes from a repository
! wget https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/master/csv/estados.csv

# add column with the state code

# read file with 'uf' as an index_col 
# uf_code = pd.read_csv('estados.csv', index_col=['uf'])

# export to a dictionary
# uf_code_dict = uf_code[['codigo_uf']].to_dict()

# colect dict items
# uf_code_dict.items()

# I've copied and pasted
# think about something more useful next time
uf_code_dict = {'RO': 11, 
                'AC': 12, 
                'AM': 13, 
                'RR': 14, 
                'PA': 15, 
                'AP': 16, 
                'TO': 17, 
                'MA': 21, 
                'PI': 22, 
                'CE': 23, 
                'RN': 24, 
                'PB': 25, 
                'PE': 26, 
                'AL': 27, 
                'SE': 28, 
                'BA': 29, 
                'MG': 31, 
                'ES': 32, 
                'RJ': 33, 
                'SP': 35, 
                'PR': 41, 
                'SC': 42, 
                'RS': 43, 
                'MS': 50, 
                'MT': 51, 
                'GO': 52, 
                'DF': 53}



--2019-06-17 17:19:00--  https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/master/csv/estados.csv
Resolving raw.githubusercontent.com... 151.101.92.133
Connecting to raw.githubusercontent.com|151.101.92.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 475 [text/plain]
Saving to: ‘estados.csv.2’

estados.csv.2       100%[===================>]     475  --.-KB/s    in 0s      

2019-06-17 17:19:00 (19.7 MB/s) - ‘estados.csv.2’ saved [475/475]



In [6]:
# FEATURE EXTRACTION

# split the data from title to get the city name
df['NomeMun'] = [df.title[i].split('Câmara Municipal de ')[1] 
                for i in range(0, len(df.link))]

# get the uf name from search from a re pattern within df.address
df['uf'] = df.address.apply(search_uf).str.split(',')
# split the CEP from the name of the state by getting the first value of each cell
df['uf'] = [df.uf[i][0] for i in range(0, len(df.uf))]
# swift state name to state code
df['uf'] = df.uf.map(uf_dict_2)

# map the state code from state initials
df['uf_code'] = df.uf.map(uf_code_dict)
# multiply the code by 1^7 to match the IBGE format
df['uf_code_multi'] = df.uf_code.multiply(1000000)

# get the street name
df['rua_camara'] = [i[1] for i in df.address.str.split(',')]

# get the street number
df['numero_camara'] = df.address.apply(search_number)

# get the zipcode
df['cep'] = [i[1] for i in df.address.str.split('CEP')]
df['cep'] = [i[1] for i in df.cep.str.split(',')]

# get the email
df['email'] = df.url_camara.apply(search_email)

# get first link from each town at interlegis website
df['url_interlegis'] = [df.link[i].split('#')[0] 
                    for i in range(0, len(df.link))]

# get the legislative house
df['url_legislativo'] = df.url_camara.apply(search_url)

# creating match var to merge pop_info file
df['match_var'] = df.NomeMun + '_' + df.uf

# clean data & NA values

In [7]:
# drop columns
df = df.drop(['url_camara', 'address', 'link'], axis=1)

In [8]:
# NA values
import numpy as np
# df = df.replace('', np.NaN)

In [9]:
# import dataset with IBGE codes

city_pop_info = pd.read_csv('/Users/liviaclarete/gdrive/code/municipios e estados/pop_gdp.csv')

city_pop_info_2016 = city_pop_info[city_pop_info['Ano']==2016]

city_code = city_pop_info_2016[['NomeMun', 'UF', 'CodMun']]

In [10]:
city_code['match_var'] = city_code.NomeMun + '_' + city_code.UF

/Users/liviaclarete/.pyenv/versions/dataAnalysis/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [11]:
# reset index
city_code = city_code.reset_index()
city_code = city_code.drop('index', axis=1)

In [12]:
# merge city_code with df
url_interlegis = city_code.merge(df, how='inner')

In [13]:
url_interlegis.shape

(5536, 14)

In [14]:
url_interlegis.url_legislativo.notnull().sum()

5536

In [15]:
url_interlegis.to_csv('url_interlegis.csv', index=False)